In [3]:
from langchain_openai import ChatOpenAI

_llm = ChatOpenAI(
    base_url="http://192.168.10.13:60001/v1",
    model="qwen2.5:7b",
    api_key="ollama"
)

In [4]:
from table_agent import Table
from info_agent import Info
from exec_agent import Exec

_table = Table(_llm)
_info = Info(_llm)
_exec = Exec(_llm)

In [5]:
from typing_extensions import TypedDict
from typing import Annotated
from operator import add

class DataBaseState(TypedDict):
     query:str
     table_names:list[str]
     infos:Annotated[list[str],add]
     table_name:str
     result:str


In [6]:
def _table_node(state):
    return _table({})


In [7]:
def _info_node(state):
    _table_name = state["table_name"]
    _rt = _info({"table_name",_table_name})
    return {"infos":[_rt]}

In [8]:
def _exec_node(state):
    _infos = state["infos"]
    _query = state["query"]
    _rt = _exec({"query":_query,"infos":_infos})
    return {"result":_rt}

In [9]:
from langgraph.types import Send

def _dispatch_table_names(state):
    sends = []
    for _table_name in state["table_names"]:
        sends.append(Send("_info_node",{"table_name":_table_name}))
    return sends

In [10]:
from langgraph.graph import StateGraph,START,END

_builder = StateGraph(DataBaseState)

_builder.add_node("_table_node",_table_node)
_builder.add_node("_info_node",_info_node)
_builder.add_node("_exec_node",_exec_node)

_builder.add_edge(START,"_table_node")
_builder.add_conditional_edges("_table_node",_dispatch_table_names)
_builder.add_edge("_info_node","_exec_node")
_builder.add_edge("_exec_node",END)

_graph = _builder.compile()



In [11]:
_rt = _graph.invoke({"query":"我们公司员工的平均年龄是多少"})
_rt

{'query': '我们公司有多少员工',
 'table_names': ['.departments', 'employees'],
 'infos': ['',
  '根据查询结果，表`employees`的结构如下：\n\n- 字段 `id`，整型字段，不是主键（`INTEGER`, not NULL）。\n- 字段 `name`，文本型字段。\n- 字段 `age`，整型字段，可能是一个表示年龄的实际年份或虚拟序数（`INTEGER`, 但在这里显示为可能的NULL值; 不是主键，无约束）。\n- 字段 `gender`，文本型字段.\n- 字段 `position` ，文本型字段\n- 字段 `department_id`，整型字段, 不是主键\n\n我们获取了所有的信息包括所有字段。需要注意的是，字段类型可能是误报或需要确认如“age”是否应该是一个序数类型的表示年龄。以下是展示该结构的查询结果：\n- `id` 的数据类型为： Integer; 其主键索引设置为 1（它代表这是一个主键）。\n- `name` 的数据类型为: Text。\n- `age` 的列数据类型标记为 Integer ;但需手动检查其具体使用情况(以确认是否需要存储实际年龄或是一个序数等），目前没有约束。\n- `gender`的 数据型为Text;\n- `position`的字段的数据类型亦设置为Text，\n- `department_id` 的数据类型也为 Integer 但是它也不是主键。 \n\n此回复不带表名字的原因是因为这个pragma查询只针对列定义，而不包含表名作为其输出。为了给出该信息，你或许期望的是使用“PRAGMA TABLE_INFO”或者类似的技术。表名为 ‘employees’。'],
 'result': '我们公司共有22名员工。'}